In [1]:
from replay_buffer2 import  ReplayBuffer
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.distributions import Categorical


In [2]:
import gym
from model import QNetwork, RNetwork

In [3]:
env = gym.make("LunarLander-v2")

In [4]:
memory = ReplayBuffer((8,), (4,), 20000, 'cuda')
memory.load_memory("expert_policy/")
memory.idx = 20000
batch_size = 64

In [5]:
obses, actions, rewards, next_obses, not_dones_no_max = memory.sample(batch_size)

In [6]:
q_shift_local = QNetwork(8, 4, 0).to('cuda')

In [7]:
reward_est = RNetwork(8,4,0).to('cuda')

In [8]:
s = obses[0]
a = actions[0]
q_shift_local(s,a)

tensor([-0.1602,  0.0845, -0.0957, -0.0351], device='cuda:0',
       grad_fn=<AddBackward0>)

In [14]:
reward_est(s,a)

tensor([0.0133], device='cuda:0', grad_fn=<AddBackward0>)

In [18]:
policy = PolicyNetwork(8,4,seed=1)

In [15]:
reward_est(s,a)

tensor([0.0133], device='cuda:0', grad_fn=<AddBackward0>)

In [17]:

class PolicyNetwork(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(PolicyNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        logits = torch.softmax(x, dim=1)
        return logits



    def sample_action(self, state):
        output = self.forward(x)
        dis = Categorical(output)
        action = dis.sample()
        log_probs  = dis.log_prob(action)
        ent = dis.entropy()
        return output, action, log_probs, ent


In [37]:
action_0 = torch.Tensor(1)
action_1 = torch.Tensor(1)
action_2 = torch.Tensor(1)
action_3 = torch.Tensor(1)
all_actions = [action_0, action_1, action_2, action_3]

In [40]:
action_0  = action_0 * 0
action_1 = action_1 * 0 + 1
action_2 = action_2 * 0 + 2
action_3 = action_3 * 0 + 3

action_0, action_1, action_2, action_3

(tensor([-0.]), tensor([1.]), tensor([2.]), tensor([3.]))

In [23]:
for idx, action in enumerate(all_actions):
    print(idx, action)

0 tensor([])
1 tensor([-1.2245e+33])
2 tensor([-1.3333e-06,  3.0716e-41])
3 tensor([-1.2245e+33,  4.5879e-41, -1.2245e+33])
